In [0]:
# set whether normal process or being run by Elsevier supporters
permissions='fulldata' # default or fulldata
# set the project name
project_name = 'test_project_1'  # This is the folder name that you work in, e.g. 'test_project_1'

In [0]:
%run /Snippets/header_008

PROJECT DETAILS:
Project identifier: test_project_1
Cluster data access level: fulldata


EXECUTION DETAILS:
Spark version: 11.3.x-scala2.12, 
Cluster Node Type: i3.xlarge, 
Driver Node Type: c4.2xlarge, 
Worker Node Type: i3.xlarge


DATA SNAPSHOT DETAILS:
This notebook uses the latest snapshot date for this version: "v008.20221021151538/", (21 October 2022)

 The following are the Scopus snapshot dates available for datasets at v008:
['v008.20220314092239/', 'v008.20220906063204/', 'v008.20221021151538/']
(If you want to use a different snapshot, in the first cell of your notebook, set the variable `custom_snapshot_date` to one of the options listed above (string data type))



DATA SAMPLE DETAILS:
Setting paths to run on full datasets (100% Scopus publication volume)


In [0]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number, coalesce, countDistinct

In [0]:
# load the datasets
#df_ani = spark.read.format("parquet").load(basePath + tablename_ani)
# EID(long), Year(int), DOI
# Af.affiliation_ids: .select(func.col('Au.email_sha1'))

df_ipr = spark.read.format("parquet").load(basePath + tablename_ipr)
# afid, preferred_name, org_type, postal_code, city, state, country, country_tag

#df_smc_ani = spark.read.format("parquet").load(basePath+tablename_smc_complete)
# EID, subfield_article, subfield_hybrid, subfield_journal

use the table 'static_data.scival_institutions_20200304' to get additional parent data

In [0]:
table_scival = table('static_data.scival_institutions_20200304')
df_scival = table_scival.filter(table_scival.country_code=='usa')
df_scival.display()

institution_id,country_num,name,region,inst_type,latitude,longitude,country_code,affiliation_ids
217032,840,National Defense University,NAM,Academic,38.866776,-77.01674,usa,"List(60021549, 60031754, 60077300)"
508001,840,Alabama A and M University,NAM,Academic,34.7839,-86.5729,usa,List(60008740)
508002,840,Albany Medical College,NAM,Academic,42.653,-73.777,usa,"List(60007977, 60021390, 108119631)"
508003,840,American University Washington DC,NAM,Academic,38.938,-77.089,usa,"List(60020420, 60116324)"
508004,840,Appalachian State University,NAM,Academic,36.2136,-81.6843,usa,List(60020441)
508006,840,Arizona State University,NAM,Academic,33.609,-113.56,usa,"List(60003699, 60003892, 60004372, 60004683, 60009043, 60012562, 60015864, 60086304, 60093724, 60093787, 60093807, 60093810, 60116379, 60116796)"
508007,840,Auburn University,NAM,Academic,32.603,-85.486,usa,"List(60001085, 60009974, 60010469, 60011754, 60013674, 60018384, 60023694, 60031288, 60075002)"
508008,840,Ball State University,NAM,Academic,40.2051,-85.404,usa,List(60028244)
508009,840,Baylor College of Medicine,NAM,Academic,29.711,-95.397,usa,"List(60000278, 60001793, 60002441, 60008978, 60009295, 60017203, 60018083, 60027347, 60107844, 60107846, 100750453, 101332385, 115881309)"
508010,840,Baylor University,NAM,Academic,31.5487,-97.1144,usa,List(60011278)


In this table affliation_ids contain the affliation belong to the same institution, we will export this data of this table and manually tagged which afid is the parent of another in this table

In [0]:
df_scival.count()

Out[11]: 3271

In [0]:
df_scival_1 = df_scival.select(
    "name", "inst_type", "country_code", func.explode(df_scival.affiliation_ids).alias("afid")
)
df_scival_1.display()

name,inst_type,country_code,afid
National Defense University,Academic,usa,60021549
National Defense University,Academic,usa,60031754
National Defense University,Academic,usa,60077300
Alabama A and M University,Academic,usa,60008740
Albany Medical College,Academic,usa,60007977
Albany Medical College,Academic,usa,60021390
Albany Medical College,Academic,usa,108119631
American University Washington DC,Academic,usa,60020420
American University Washington DC,Academic,usa,60116324
Appalachian State University,Academic,usa,60020441


In [0]:
df_scival_1.count()

Out[13]: 7880

In [0]:
df_scival_1.select(countDistinct('afid')).show()

+--------------------+
|count(DISTINCT afid)|
+--------------------+
|                7870|
+--------------------+



In [0]:
# after distinct count, we can see 7880 become 7870, check this special affid

df_groupby = df_scival_1.groupby('afid').count().withColumnRenamed("afid",'affid')
df_toshow = df_groupby.join(df_scival_1, df_groupby.affid==df_scival_1.afid,"left").drop('afid').groupby(df_groupby.columns).agg(func.collect_set("name").alias('name_list'))
df_toshow.sort('count', ascending=False).display()

affid,count,name_list
60013881,3,"List(University of Chicago, United States Department of Energy, Fermi National Accelerator Laboratory)"
60018020,3,"List(Princeton University, United States Department of Energy, Princeton Plasma Physics Laboratory)"
60025590,3,"List(Stanford University, United States Department of Energy, Stanford Linear Accelerator Center)"
60028609,3,"List(University of Chicago, Argonne National Laboratory, United States Department of Energy)"
60008417,2,"List(Beckman Coulter, Danaher)"
60112067,2,"List(Ultragenyx Pharmaceutical Inc, Ultragenyx Pharmaceutical)"
6000278,1,"List(Becton, Dickinson and Company)"
6002494,1,List(Bausch & Lomb Incorporated)
60000000,1,List(University of California at Davis)
60000001,1,List(Walter Reed Army Institute of Research)


In [0]:
df_scival_0 = df_scival.select("name","affiliation_ids").withColumn('preferred_name',func.col('name'))
af_data = df_ipr.filter(df_ipr.country_tag=='usa')

In [0]:
# use the insitution name to try to get the afid in df_ipr which contains afid data
merge_0 = df_scival_0.join(af_data,'preferred_name').select('preferred_name','name','affiliation_ids','afid','postal_code','org_type','parent','parent_preferred_name')
merge_0.display()

preferred_name,name,affiliation_ids,afid,postal_code,org_type,parent,parent_preferred_name
National Bureau of Economic Research,National Bureau of Economic Research,"List(60020337, 114289419)",103840730,27858-4353,null,60016280,East Carolina University
Kennedy and Jenks Consultants,Kennedy and Jenks Consultants,List(60096337),60096337,85012,comp,null,null
American Psychological Association,American Psychological Association,List(60012920),118346689,null,null,null,null
Texas State University,Texas State University,List(60030452),60030452,78666-4684,univ,null,null
National Research Council,National Research Council,"List(60009235, 60098160)",116073221,null,null,null,null
Sealed Air Corporation,Sealed Air Corporation,List(60009644),122660056,53177,null,null,null
National Research Council,National Research Council,"List(60009235, 60098160)",100894154,null,null,null,null
Washington State University Tri-Cities,Washington State University Tri-Cities,List(60016326),60016326,99354-1671,univ,null,null
University of Alabama,University of Alabama,List(60025371),122980350,27695-0001,null,60004923,NC State University
United States Department of Agriculture,United States Department of Agriculture,"List(60000635, 60000807, 60000878, 60001145, 60001243, 60001283, 60001518, 60001609, 60001963, 60001989, 60002110, 60002418, 60002603, 60002722, 60002942, 60003342, 60003518, 60003888, 60003922, 60004653, 60005305, 60005850, 60005864, 60006180, 60006631, 60006914, 60007602, 60007681, 60007849, 60007901, 60008135, 60008528, 60008574, 60008671, 60008929, 60009224, 60009911, 60010048, 60010372, 60010621, 60010639, 60010838, 60011043, 60011435, 60011959, 60012076, 60013171, 60013275, 60013351, 60013741, 60014987, 60014998, 60016024, 60016539, 60017210, 60017654, 60018068, 60018268, 60018907, 60019520, 60019986, 60020010, 60020049, 60020199, 60021335, 60021683, 60022061, 60022088, 60022218, 60022541, 60022600, 60022772, 60023175, 60023249, 60023624, 60023722, 60023830, 60024862, 60025189, 60025292, 60025549, 60025724, 60026143, 60026170, 60026715, 60027566, 60027678, 60028226, 60028521, 60028657, 60029237, 60029951, 60030366, 60030463, 60030965, 60031032, 60032280, 60032384, 60032434, 60032832, 60032935, 60071509, 60071534, 60071535, 60071536, 60075868, 60076225, 60076941, 60076945, 60083179, 60086585, 60090887, 60090910, 60090934, 60091232, 60091246, 60091302, 60092179, 60093609, 60101260, 60103048, 60103638, 60103646, 60103647, 60103648, 60103649, 60103650, 60103651, 60103652, 60103675, 60103678, 60103685, 60103775, 60110412)",103568455,null,null,60010372,USDA ARS Moscow Forestry Sciences Lab


In [0]:
merge_0.count()

Out[18]: 10546

In [0]:
# use the afid to get the affiliation information for mannual check
merge_1 = df_scival_1.join(af_data,'afid').select('afid','name','inst_type','preferred_name','postal_code','city','org_type',"parent","parent_preferred_name")
merge_1.display()

afid,name,inst_type,preferred_name,postal_code,city,org_type,parent,parent_preferred_name
60005457,Seagate Technology,Corporate,Seagate Research,15222,Pittsburgh,comp,null,null
60029888,University of North Dakota,Academic,UND School of Medicine & Health Sciences - Southwest Campus,58202-1975,Bismarck,meds,null,null
60096337,Kennedy and Jenks Consultants,Corporate,Kennedy and Jenks Consultants,85012,Phoenix,comp,null,null
60006816,University of Pittsburgh,Academic,Safar Center for Resuscitation Research,15224-1334,Pittsburgh,resi,null,null
60015232,Yale University,Academic,Yale-New Haven Hospital,06510,New Haven,hosp,null,null
60030452,Texas State University,Academic,Texas State University,78666-4684,San Marcos,univ,null,null
60101400,Texas A and M University,Academic,Texas Forest Service,77840-7896,College Station,resi,null,null
60028870,Eastern Virginia Medical School,Academic,The Virginia Consortium in Clinical Psychology,23453,Virginia Beach,meds,null,null
60003283,Wisconsin Energy Corporation (We Energies),Corporate,Wisconsin Energy Corporation,53203,Milwaukee,comp,null,null
60016326,Washington State University Tri-Cities,Academic,Washington State University Tri-Cities,99354-1671,Richland,univ,null,null


In [0]:
#this notebook finally export these two tables for mannually tag the affid parent of the institutions
df_export_1 = merge_0
df_export_2 = merge_1